[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [3]:
# Read dataset with extra params sep='\t', encoding="latin-1"
data = pd.read_csv('../input/traindata/labeledTrainData.tsv', sep='\t', encoding="latin-1")
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


## 2. Preprocessing

In [8]:
# stop words
from collections import Counter

vocab = Counter()
for review in data.review:
    for word in review.split(' '):
        vocab[word] += 1

vocab.most_common(10)

[('the', 287025),
 ('a', 155092),
 ('and', 152651),
 ('of', 142970),
 ('to', 132568),
 ('is', 103227),
 ('in', 85576),
 ('I', 65971),
 ('that', 64555),
 ('this', 57195)]

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop_words' 
# and save them in vocab_reduced
for w, c in vocab.items():
    if not w in stop_words:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[('I', 65971),
 ('/><br', 50935),
 ('The', 33760),
 ('movie', 30496),
 ('film', 27394),
 ('one', 20685),
 ('like', 18133),
 ('This', 12279),
 ('would', 11922),
 ('good', 11435),
 ('It', 10950),
 ('really', 10814),
 ('even', 10605),
 ('see', 10154),
 ('-', 9355),
 ('get', 8776),
 ('story', 8523),
 ('much', 8506),
 ('time', 7762),
 ('make', 7485)]

In [11]:
# Removing special characters and "trash"
import re
def preprocessor(text):
     # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [15]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [16]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split

X = data['review']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=80)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocessor at 0x7f456df0ca70>,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function tokenizer_porter at 0x7f456de95050>)),
                ('clf', LogisticRegression(random_state=0))])

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)

## 4. Evaluate Model

In [23]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.8869333333333334
confusion matrix:
 [[3239  447]
 [ 401 3413]]
classification report:
               precision    recall  f1-score   support

           0       0.89      0.88      0.88      3686
           1       0.88      0.89      0.89      3814

    accuracy                           0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



In [25]:
twits = [
    "I do not feel not bad", # Phuc +1
    'This model is "so good" :))', # Long -1
    'we are who we are', # Nghi 0
    'its good to be bad sometimes', # PA +1
    'what a wonderful failure! (sarcasm :)))', #Phuc +1
    'People do not like the bad things', # Chi 0
    'We finally have the test result. You are positive', # Long +1
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

I do not feel not bad --> Negative, Positive = [0.96959996 0.03040004]
This model is "so good" :)) --> Negative, Positive = [0.2286281 0.7713719]
we are who we are --> Negative, Positive = [0.49384433 0.50615567]
its good to be bad sometimes --> Negative, Positive = [0.76736132 0.23263868]
what a wonderful failure! (sarcasm :))) --> Negative, Positive = [0.27667535 0.72332465]
People do not like the bad things --> Negative, Positive = [0.96624158 0.03375842]
We finally have the test result. You are positive --> Negative, Positive = [0.65766122 0.34233878]


## 5. Export Model 

In [26]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open('logisticRegression.pkl', 'wb'))

In [ ]:
with open('logisticRegression.pkl', 'rb') as model:
    reload_model = pickle.load(model)
preds = reload_model.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')